### Bert-base raw model

In [32]:

import pandas as pd
import numpy as np
import torch
import transformers

from transformers import BertForMaskedLM
from transformers import BertTokenizer
from transformers import BertModel

In [33]:
# HYPERPARAMS
SEED_SPLIT = 0
SEED_TRAIN = 0

MAX_SEQ_LEN = 128
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 16
LEARNING_RATE = 2e-5 
LR_WARMUP_STEPS = 100
WEIGHT_DECAY = 0.01

In [83]:
MODEL = 'bert'
bert_type = 'bert-base-uncased'
TokenizerClass = BertTokenizer
# ModelClass = BertForMaskedLM 
ModelClass = BertModel 



tokenizer = TokenizerClass.from_pretrained(
            bert_type, use_fast=True, do_lower_case=False, max_len=MAX_SEQ_LEN
            )
model = ModelClass.from_pretrained(bert_type)


# tokenizer = TokenizerClass.from_pretrained(
#             './your_path/model', use_fast=True, do_lower_case=False, max_len=MAX_SEQ_LEN
#             )
# model = ModelClass.from_pretrained('./your_path/model')


model.eval()


# tokenizer = BertTokenizer.from_pretrained('data/cased_L-12_H-768_A-12') #改为自己存放模型的目录
# model = BertModel.from_pretrained('data/cased_L-12_H-768_A-12')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [84]:

# vocab = open('./your_path/model/vocab.txt', 'r')
vocab = open('/Users/wshuo/Desktop/小论文/bert/uncased_L-12_H-768_A-12/vocab.txt', 'r')
vocab_wds = [line.strip('\n') for line in vocab]

def encode_phrase(phrase):
#     phrase = "Your English phrase goes here."
    tokens = tokenizer.encode_plus(phrase, add_special_tokens=True, return_tensors='pt')
#     print(tokens['input_ids'])
    ids = tokens['input_ids'].numpy().tolist()
    id_char = ""
    for i in ids[0]:
        id_char += "{0}({1}), ".format(i, vocab_wds[i])
    print(id_char)
    with torch.no_grad():
        outputs = model(tokens['input_ids'], tokens['attention_mask'])
        phrase_vector = outputs.last_hidden_state.mean(dim=1).squeeze()
    return phrase_vector

    
# print(phrase_vector.shape)

In [85]:
vec = encode_phrase("rplidar")
# print(vec)

101([CLS]), 1054(r), 24759(##pl), 8524(##ida), 2099(##r), 102([SEP]), 


In [70]:
def cos_similar(sen_a_vec, sen_b_vec):
    vector_a = np.mat(sen_a_vec)
    vector_b = np.mat(sen_b_vec)
    num = float(vector_a*vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num/denom
    return cos


In [71]:
def embedding_similarity(p1,p2):
    v1 = encode_phrase(p1)
    v2 = encode_phrase(p2)
    sim = cos_similar(v1, v2)
    return sim

In [72]:
s1 = "rplidar"
s2 = "The rplidar ros package support rplidar A2/A1"
# tensor([[  101,   100,  1054, 24759,  8524(##ida),  2099, 20996,  2015,  7427,  2490,
#           1054, 24759,  8524,  2099,   100,  1013,   100,   102]])
print(embedding_similarity(s1, s2))
s2 = "rospeex"
print(embedding_similarity(s1, s2))
s1 = "turtlebot"
s2 = "turtlebot2"
s3 = "turtlebot3"
print(embedding_similarity(s1, s2))
print(embedding_similarity(s2, s1))
print(embedding_similarity(s1, s3))
print(embedding_similarity(s2, s3))

tensor([[  101,  1054, 24759,  8524,  2099,   102]])
tensor([[  101,   100,  1054, 24759,  8524,  2099, 20996,  2015,  7427,  2490,
          1054, 24759,  8524,  2099,   100,  1013,   100,   102]])
0.6360173070810841
tensor([[  101,  1054, 24759,  8524,  2099,   102]])
tensor([[  101, 20996, 13102,  4402,  2595,   102]])
0.7411139171936202
tensor([[  101, 13170, 18384,   102]])
tensor([[  101, 13170, 18384,  2475,   102]])
0.8477706363141795
tensor([[  101, 13170, 18384,  2475,   102]])
tensor([[  101, 13170, 18384,   102]])
0.8477706363141795
tensor([[  101, 13170, 18384,   102]])
tensor([[  101, 13170, 18384,  2509,   102]])
0.8476976483004389
tensor([[  101, 13170, 18384,  2475,   102]])
tensor([[  101, 13170, 18384,  2509,   102]])
0.9450158355222025
